This notebook provides an regular expression based approach to search for letter openings in a literary text corpus. The first part of the notebook covers the preparation of the corpus. The second part is the extraction of letter openings by looking for typical expressions in the beginning of German-language letters. The openings are collected in a list. The algorithms loops over that list to see if a list entry matches with a pattern in the corpus texts. The list can be modified. Please take into account that too generic letter openings like r'Mein[e]' cannot be processed because there are too many matches in the corpus texts (the most of them false positives).

In [190]:
# Import 
import os
import pandas as pd
import regex as re
from pathlib import Path
from collections import Counter
import csv


## Read in the corpus

In the following, we read in the corpus. Here, we only take a subset of the corpus to shorten the process time.  

In [191]:
# Generate a corpus by loading all the txt files from the chosen directory 
# and list the names of the first 10 txt files 
#corpus = os.listdir('d-prose_subsets/d-prose_subset2')
corpus = os.listdir('d-prose_subset2')
corpus[:10]

['Fontane_Theodor_Cecile.txt',
 'Hollaender_Felix_Die_Briefe_des_Fraeulein.txt',
 'Heyse_Paul_Gegen_den_Strom.txt',
 'Federer_Heinrich_Umbrische_Reisegeschichtlein_San_Benedettos_Dornen.txt',
 'Groller_Balduin_Detektiv_Dagobert_Eine_teure_Depesche.txt',
 'Heyse_Paul_Marienkind.txt',
 'Frobenius_Leo_Das_schwarze_Dekameron_Siga_Sanke.txt',
 'Ganghofer_Ludwig_Auf_der_Wallfahrt.txt',
 'Janitschek_Maria_Das_kleine_Huendchen.txt',
 'Frapan_Ilse_Thedche_Bolzen.txt']

With the next cell, we ask for the number of corpus texts in the chosen subset. 

In [192]:
# Print how many txt files are in the corpus
corpus_length = len(corpus)
print(corpus_length)

494


## Convert the corpus to a dataframe 

We then create an empty dictionary and add the file name and the text of the document as columns to build a dataframe of two columns.

In [193]:
# Create an empty dictionary for preparation of the conversion of the txt-file-corpus to a data frame
empty_dictionary = {}

# Loop through the folder of documents to open and read each one
for document in corpus:
    with open('d-prose_subsets/d-prose_subset2/' + document, 'r', encoding = 'utf-8') as to_open:
         empty_dictionary[document] = to_open.read()

# Populate the data frame with two columns: file name and document text
d_prose_texts = (pd.DataFrame.from_dict(empty_dictionary, 
                                       orient = 'index')
                .reset_index().rename(index = str, 
                                      columns = {'index': 'file_name', 0: 'document_text'}))

In the next cell, we verify the content of the first 10 lines of the dataframe.

In [194]:
# show the first 10 lines of the data frame
d_prose_texts[:10]

,file_name,document_text
0,Fontane_Theodor_Cecile.txt,Cécile\n\nErstes Kapitel\n\n»Thale. Zweiter…«\...
1,Hollaender_Felix_Die_Briefe_des_Fraeulein.txt,"Die Briefe des Fräulein Brandt\n\nIserbaude, 7..."
2,Heyse_Paul_Gegen_den_Strom.txt,Gegen den Strom\n\nErstes Kapitel.\n\nEs war z...
3,Federer_Heinrich_Umbrische_Reisegeschichtlein_...,San Benedettos Dornen und San Francescos Rosen...
4,Groller_Balduin_Detektiv_Dagobert_Eine_teure_D...,Eine teure Depesche\n\nSie saßen wieder zu dri...
5,Heyse_Paul_Marienkind.txt,"Marienkind\n\nAuf der Landstraße, die in gerin..."
6,Frobenius_Leo_Das_schwarze_Dekameron_Siga_Sank...,Siga Sanke\n\nSiga Sanke wohnte in dem Dorfe S...
7,Ganghofer_Ludwig_Auf_der_Wallfahrt.txt,Auf der Wallfahrt\n\nWieder war ein Schächtelc...
8,Janitschek_Maria_Das_kleine_Huendchen.txt,"Das kleine Hündchen\n\n1.\n\n»Wunderlich, wund..."
9,Frapan_Ilse_Thedche_Bolzen.txt,»Thedche Bolzen«\n\nSie sieht nicht eben hübsc...


In the next cell, we extract the title of the text by extracting the first line followed by two line breaks. This is possible because we know about the structure of the text that were manually prepared following that schema.

In [195]:
#extract the title of the text as a further column with metadata


# Define the regular expression pattern to extract the title followed by double line break \n\n
pattern = r'^(.*?)\n\n'

# Extract the first line and create a new 'titles' column
d_prose_texts['title'] = d_prose_texts['document_text'].str.extract(pattern, flags=re.DOTALL)

# Print the DataFrame to see the results
d_prose_texts[:10]


,file_name,document_text,title
0,Fontane_Theodor_Cecile.txt,Cécile\n\nErstes Kapitel\n\n»Thale. Zweiter…«\...,Cécile
1,Hollaender_Felix_Die_Briefe_des_Fraeulein.txt,"Die Briefe des Fräulein Brandt\n\nIserbaude, 7...",Die Briefe des Fräulein Brandt
2,Heyse_Paul_Gegen_den_Strom.txt,Gegen den Strom\n\nErstes Kapitel.\n\nEs war z...,Gegen den Strom
3,Federer_Heinrich_Umbrische_Reisegeschichtlein_...,San Benedettos Dornen und San Francescos Rosen...,San Benedettos Dornen und San Francescos Rosen
4,Groller_Balduin_Detektiv_Dagobert_Eine_teure_D...,Eine teure Depesche\n\nSie saßen wieder zu dri...,Eine teure Depesche
5,Heyse_Paul_Marienkind.txt,"Marienkind\n\nAuf der Landstraße, die in gerin...",Marienkind
6,Frobenius_Leo_Das_schwarze_Dekameron_Siga_Sank...,Siga Sanke\n\nSiga Sanke wohnte in dem Dorfe S...,Siga Sanke
7,Ganghofer_Ludwig_Auf_der_Wallfahrt.txt,Auf der Wallfahrt\n\nWieder war ein Schächtelc...,Auf der Wallfahrt
8,Janitschek_Maria_Das_kleine_Huendchen.txt,"Das kleine Hündchen\n\n1.\n\n»Wunderlich, wund...",Das kleine Hündchen
9,Frapan_Ilse_Thedche_Bolzen.txt,»Thedche Bolzen«\n\nSie sieht nicht eben hübsc...,»Thedche Bolzen«


You can see still some \n\n these are line breaks. You can use regular expressions to extract the first line as title of the text. And the file name contains the name of the author, but that is not as easy to extract. Better to be extracted from the metadata with a comparison of filename and filename indicated in metadata.

In the next cell, we do some basic text cleaning steps.

## Preprocessing

In [196]:
#create a new column
#use regular expressions to clean the plain text and store the cleaned text in a new column as a further layer of the text without deleting the original version
d_prose_texts['clean_text'] = d_prose_texts['document_text'].str.replace('\s+', ' ') # remove double white space
d_prose_texts['clean_text'] = d_prose_texts['clean_text'].str.replace('\n+', '\n') # remove double line break
d_prose_texts['clean_text'] = d_prose_texts['clean_text'].str.replace('&', 'and') # exchange & for 'and'



/var/folders/qf/cmgr81q1733_dshx90_j6r9c0000gp/T/ipykernel_3853/219875720.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  d_prose_texts['clean_text'] = d_prose_texts['document_text'].str.replace('\s+', ' ') # remove double white space
/var/folders/qf/cmgr81q1733_dshx90_j6r9c0000gp/T/ipykernel_3853/219875720.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  d_prose_texts['clean_text'] = d_prose_texts['clean_text'].str.replace('\n+', '\n') # remove double line break


In [197]:
# show the first 10 lines of the data frame
d_prose_texts[:10]

,file_name,document_text,title,clean_text
0,Fontane_Theodor_Cecile.txt,Cécile\n\nErstes Kapitel\n\n»Thale. Zweiter…«\...,Cécile,Cécile Erstes Kapitel »Thale. Zweiter…« »Letzt...
1,Hollaender_Felix_Die_Briefe_des_Fraeulein.txt,"Die Briefe des Fräulein Brandt\n\nIserbaude, 7...",Die Briefe des Fräulein Brandt,"Die Briefe des Fräulein Brandt Iserbaude, 7. J..."
2,Heyse_Paul_Gegen_den_Strom.txt,Gegen den Strom\n\nErstes Kapitel.\n\nEs war z...,Gegen den Strom,Gegen den Strom Erstes Kapitel. Es war zu Anfa...
3,Federer_Heinrich_Umbrische_Reisegeschichtlein_...,San Benedettos Dornen und San Francescos Rosen...,San Benedettos Dornen und San Francescos Rosen,San Benedettos Dornen und San Francescos Rosen...
4,Groller_Balduin_Detektiv_Dagobert_Eine_teure_D...,Eine teure Depesche\n\nSie saßen wieder zu dri...,Eine teure Depesche,Eine teure Depesche Sie saßen wieder zu dritt ...
5,Heyse_Paul_Marienkind.txt,"Marienkind\n\nAuf der Landstraße, die in gerin...",Marienkind,"Marienkind Auf der Landstraße, die in geringer..."
6,Frobenius_Leo_Das_schwarze_Dekameron_Siga_Sank...,Siga Sanke\n\nSiga Sanke wohnte in dem Dorfe S...,Siga Sanke,Siga Sanke Siga Sanke wohnte in dem Dorfe Söin...
7,Ganghofer_Ludwig_Auf_der_Wallfahrt.txt,Auf der Wallfahrt\n\nWieder war ein Schächtelc...,Auf der Wallfahrt,Auf der Wallfahrt Wieder war ein Schächtelchen...
8,Janitschek_Maria_Das_kleine_Huendchen.txt,"Das kleine Hündchen\n\n1.\n\n»Wunderlich, wund...",Das kleine Hündchen,"Das kleine Hündchen 1. »Wunderlich, wunderlich..."
9,Frapan_Ilse_Thedche_Bolzen.txt,»Thedche Bolzen«\n\nSie sieht nicht eben hübsc...,»Thedche Bolzen«,»Thedche Bolzen« Sie sieht nicht eben hübsch a...


Now, we want to find out more about the texts in our corpus. 
For instance, we want to find out if there are letters in the corpus texts.

# Search for Letter Openings
The code of the next cell iterates over the corpus texts and looks for matches with the regular expressions of the list "letter_openings".

## Define the Regular Expressions to find Letter Openings

In [198]:
# Liste von Briefanfängen und -enden
letter_openings = [
    r'[»]?Mein[e]? [lL]iebe[rsn]?\s[A-Za-z]*[!]',
    r'[»]?Hochverehrte[rsn]?\s[A-Za-z]*[!]',
    r'[»]?Einziggeliebt[er]?\s[A-Za-z]*[.!,]?',
    r'[»]?Geehrte[rsn]?\s[A-Za-z]*[.!,]?',
    r'[»]?Sehr geehrte[srn]?\s[A-Za-z]*[.!,]?',
    r'[»]?Sehr verehrte[rs]?\s[A-Za-z]*[.!,]?',
    r'[»]?Grüss dich\s[A-Za-z]*[.!,]?',
    r'[»]?Grüß dich\s[A-Za-z]*[.!,]?',
    #r'[»]?\b(?!die\s+)Liebe\b\s[A-Za-z]*[.!,]?',
    #r'[»]?Liebe[sr]+\s[A-Za-z]*[.!,]?',
    r'»Liebe[sr]+\s[A-Z][a-z\s]*[A-Za-z!]*',
    r'»Liebste[sr]?\s[A-Za-z]*[A-Za-z!]*',
    r'[»]?Lieber Vater[!]',
    r'[»]?Liebster Vater[!]',
    r'[»]?Liebe Mutter[!]',
    r'[»]?Liebste Mutter[!]',
    r'[»]?Lieber Freund[!]',
    r'[»]?Geliebteste[r]?[!]',
    r'[»]?Geliebte[r][!]',
    r'[»]?Geliebte[!]',
    r'[»]?Einzig geliebteste[r]?[.!,]?',
    r'[»]?Einzig geliebte[r]?[.!,]?',
    r'[»]?Hochverehrter Herr[.!,]?',
    r'[»]?Hochverehrte Frau[.!,]?',
    r'[»]?Werte[rs]?\s[A-Za-z]*[!]',
    r'[»]?Mein geliebte[rs]?\s[A-Za-z]*[.!,]?',
    r'[»]?Teuerste[sr]?\s[A-Za-z]*[.!,]?',
    r'[»]?Teure[sr]?\s[A-Za-z]*[.!,]?',
    r'[»]?Mein Liebchen[.!,]?',
    r'(?:[A-ZÄÖÜa-zäöüß\s]+,\s*den\s+\d+\.\s+[A-ZÄÖÜa-zäöüß]+\s+\d{4}\.)',
    #r'\b\d{1,2}\. (Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember)(?: \d{2,4})?\b'
    r'(?:[A-ZÄÖÜa-zäöüß\s]+,\s*den\s+\d+\.\s+[A-ZÄÖÜa-zäöüß]+\s+\d{4}\.)|\b\d{1,2}\. (Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember)(?: \d{2,4})?\b'
    # Ab hier weitere auszuschließende Fälle
]


## Do the matching and get the output as a dictionary

In [200]:
#This function is to only extract the letter openings matching with the regex.

# Function to extract the exact regex matches 
def extract_matches(text):
    extracted_matches = []
    for pattern in letter_openings:
        for match in re.finditer(pattern, text):
            match_text = match.group(0)  # Get the matched text
            match_start = match.start()  # Get the start index of the match
            
            # Append the detected text and the index of the first character to the list
            extracted_matches.append((match_text, match_start))
    
    return extracted_matches

# Apply the extraction function to each row of the DataFrame
d_prose_texts['extracted_matches'] = d_prose_texts['clean_text'].apply(extract_matches)

# Print the DataFrame to see the results
print(d_prose_texts[['clean_text', 'extracted_matches']])


                                            clean_text  \
0    Cécile Erstes Kapitel »Thale. Zweiter…« »Letzt...   
1    Die Briefe des Fräulein Brandt Iserbaude, 7. J...   
2    Gegen den Strom Erstes Kapitel. Es war zu Anfa...   
3    San Benedettos Dornen und San Francescos Rosen...   
4    Eine teure Depesche Sie saßen wieder zu dritt ...   
..                                                 ...   
489  Enzio Es war ein weites, bequem eingerichtetes...   
490  Dagoberts unfreiwillige Reise Andreas Grumbach...   
491  Die Liebe Gottes Wer auf fröhlichcr Sommerreis...   
492  Dort oben Ja, Kinder, das ist wohl so, das gla...   
493  Der blinde Passagier Wir hatten uns über das u...   

                                     extracted_matches  
0    [(»Lieber Pierre, 28539), (»Lieber Freund, 310...  
1    [(»Liebes Kind, 57567), (Teuerster wird, 16684...  
2    [(Geehrten nun, 349367), (»Mein geliebtes Herz...  
3                                                   []  
4                 

In [201]:
# Function to count the number of matches for each text
def count_matches(matches):
    return len(matches)

# Create a new column with the count of matches for each text
d_prose_texts['match_count'] = d_prose_texts['extracted_matches'].apply(count_matches)

# Print the DataFrame to see the results
print(d_prose_texts[['clean_text', 'extracted_matches', 'match_count']])

                                            clean_text  \
0    Cécile Erstes Kapitel »Thale. Zweiter…« »Letzt...   
1    Die Briefe des Fräulein Brandt Iserbaude, 7. J...   
2    Gegen den Strom Erstes Kapitel. Es war zu Anfa...   
3    San Benedettos Dornen und San Francescos Rosen...   
4    Eine teure Depesche Sie saßen wieder zu dritt ...   
..                                                 ...   
489  Enzio Es war ein weites, bequem eingerichtetes...   
490  Dagoberts unfreiwillige Reise Andreas Grumbach...   
491  Die Liebe Gottes Wer auf fröhlichcr Sommerreis...   
492  Dort oben Ja, Kinder, das ist wohl so, das gla...   
493  Der blinde Passagier Wir hatten uns über das u...   

                                     extracted_matches  match_count  
0    [(»Lieber Pierre, 28539), (»Lieber Freund, 310...            9  
1    [(»Liebes Kind, 57567), (Teuerster wird, 16684...           28  
2    [(Geehrten nun, 349367), (»Mein geliebtes Herz...            3  
3                      

In [202]:
# Function to extract matches and following 100 characters for a match

def extract_matches_and_following(text):
    extracted_matches = []
    for pattern in letter_openings:
        for match in re.finditer(pattern, text):
            match_text = match.group(0)  # Get the matched text
            match_start = match.start()  # Get the start index of the match
            
            # Extract the following 100 characters
            #match_end = match_start + len(match_text)
            following_text = text[match_start:match_start + 100]
            
            # Append the detected text and the index of the first character to the list
            extracted_matches.append((following_text, match_start))
    
    return extracted_matches

# Apply the extraction function to each row of the DataFrame
d_prose_texts['extracted_matches_context'] = d_prose_texts['clean_text'].apply(extract_matches_and_following)

# Print the DataFrame to see the results
print(d_prose_texts[['clean_text', 'extracted_matches_context']])


                                            clean_text  \
0    Cécile Erstes Kapitel »Thale. Zweiter…« »Letzt...   
1    Die Briefe des Fräulein Brandt Iserbaude, 7. J...   
2    Gegen den Strom Erstes Kapitel. Es war zu Anfa...   
3    San Benedettos Dornen und San Francescos Rosen...   
4    Eine teure Depesche Sie saßen wieder zu dritt ...   
..                                                 ...   
489  Enzio Es war ein weites, bequem eingerichtetes...   
490  Dagoberts unfreiwillige Reise Andreas Grumbach...   
491  Die Liebe Gottes Wer auf fröhlichcr Sommerreis...   
492  Dort oben Ja, Kinder, das ist wohl so, das gla...   
493  Der blinde Passagier Wir hatten uns über das u...   

                             extracted_matches_context  
0    [(»Lieber Pierre«, sagte sie dann mit sich ras...  
1    [(»Liebes Kind,« antworte ich trocken, »du bis...  
2    [(Geehrten nun überreichte. Worauf der alte Ma...  
3                                                   []  
4                 

In [203]:
d_prose_texts['extracted_matches_context'][20:70]

20                                                   []
21                                                   []
22                                                   []
23    [(Teurer Held Perikles, was sagst du zu solch'...
24                                                   []
25                                                   []
26                                                   []
27                                                   []
28    [(»Lieber Herr Doktor! Ich schicke Ihnen ein M...
29                                                   []
30                                                   []
31                                                   []
32    [(27. März!‹ – Ich aber sagte: ›Wir haben heut...
33    [(»Lieber Holk, Sie kennen doch hoffentlich di...
34    [(24. Dezember. Kein Schnee, kein Nebel, kein ...
35                                                   []
36                                                   []
37    [(»Lieber Kerl, du brauchst nur an dem Tis

## Save the dictionary to a csv file

In [206]:
#save the original huge dataframe to a new csv.file

d_prose_texts.to_csv('df_letter_openings_subset2.csv', index=False)

## Postprocessing

In [207]:

# Now look into the output and find the most common false positives.
# Collect them in this list of common false positives 
#that you can consequently use to delete the values from the received matches of letter openings to clean the output as a postprocessing step.

common_false_positives = [
    r'[»]?Lieber Gott',
    r'Liebe und',
    r'Liebe zu',
    r'Liebe von',
    r'[»]?Liebe[sr]+\s[A-Za-z,!]*[«]? [sag,unter]',
    r'Liebe [A-Za-z,\s]*[«]? [sag,unter]',
    r'Lieber [A-Za-z,\s]*[«]? [sag,unter]',
    r'Liebes [A-Za-z,\s]*[«]? [sag,unter]',
    r'Grüß dich Gott, [A-Za-z]',
    r'Teurer [A-Za-z,\s]* [sag,unter]',
    r'Hochverehrte Festversammlung!',
    r'Teuerste bedeutet',
    r'Geehrten',
    r'Grüß dich Gott, ',
    r'Grüß dich Gott!« [sag,unter]',
    r'Liebster Jesu!',
    r'Lieber Jesu!',
    r'Lieber Vogel,',
    r'Teuerste war',
    r'(?:[A-ZÄÖÜa-zäöüß\s]+,\s*den\s+\d+\.\s+[A-ZÄÖÜa-zäöüß]+\s+\d{4}\.)|\b\d{1,2}\. (Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember)(?: \d{2,4})?\b'
    
]

In [208]:


# Function to remove common false positives
def remove_common_false_positives(extracted_matches_column, common_false_positives):
    # Create a copy of the DataFrame column to avoid modifying the original
    cleaned_matches_column = extracted_matches_column.copy()

    # Iterate through the rows of the DataFrame column
    for index, row in cleaned_matches_column.iteritems():
        # Create a list to store valid matches for the current row
        valid_matches = []

        for match_text, match_start in row:
            # Check if the match_text matches any of the common_false_positives patterns
            if not any(re.search(fp_pattern, match_text) for fp_pattern in common_false_positives):
                valid_matches.append((match_text, match_start))

        # Update the DataFrame column with valid matches for the current row
        cleaned_matches_column[index] = valid_matches

    return cleaned_matches_column

# Remove common false positives
cleaned_extracted_matches = remove_common_false_positives(d_prose_texts['extracted_matches_context'], common_false_positives)

# Update the 'extracted_matches' column in the DataFrame with the cleaned matches
d_prose_texts['cleaned_matches_context'] = cleaned_extracted_matches

# Print the modified DataFrame with cleaned matches
print(d_prose_texts)


                                             file_name  \
0                           Fontane_Theodor_Cecile.txt   
1        Hollaender_Felix_Die_Briefe_des_Fraeulein.txt   
2                       Heyse_Paul_Gegen_den_Strom.txt   
3    Federer_Heinrich_Umbrische_Reisegeschichtlein_...   
4    Groller_Balduin_Detektiv_Dagobert_Eine_teure_D...   
..                                                 ...   
489                           Huch_Friedrich_Enzio.txt   
490  Groller_Balduin_Detektiv_Dagobert_Dagoberts_un...   
491  Ganghofer_Ludwig_Fliegender_Sommer_Die_Liebe_G...   
492   Frapan_Ilse_Was_der_Alltag_dichtet_Dort_oben.txt   
493  Ganghofer_Ludwig_Fliegender_Sommer_Der_blinde_...   

                                         document_text  \
0    Cécile\n\nErstes Kapitel\n\n»Thale. Zweiter…«\...   
1    Die Briefe des Fräulein Brandt\n\nIserbaude, 7...   
2    Gegen den Strom\n\nErstes Kapitel.\n\nEs war z...   
3    San Benedettos Dornen und San Francescos Rosen...   
4    Eine teu

In [211]:
# Function to count the number of matches for each text
def count_matches(matches):
    return len(matches)

# Create a new column with the count of matches for each text
d_prose_texts['cleaned_match_count'] = d_prose_texts['cleaned_matches_context'].apply(count_matches)

# Print the DataFrame to see the results
print(d_prose_texts[['clean_text', 'cleaned_matches_context', 'cleaned_match_count']])

                                            clean_text  \
0    Cécile Erstes Kapitel »Thale. Zweiter…« »Letzt...   
1    Die Briefe des Fräulein Brandt Iserbaude, 7. J...   
2    Gegen den Strom Erstes Kapitel. Es war zu Anfa...   
3    San Benedettos Dornen und San Francescos Rosen...   
4    Eine teure Depesche Sie saßen wieder zu dritt ...   
..                                                 ...   
489  Enzio Es war ein weites, bequem eingerichtetes...   
490  Dagoberts unfreiwillige Reise Andreas Grumbach...   
491  Die Liebe Gottes Wer auf fröhlichcr Sommerreis...   
492  Dort oben Ja, Kinder, das ist wohl so, das gla...   
493  Der blinde Passagier Wir hatten uns über das u...   

                               cleaned_matches_context  cleaned_match_count  
0    [(»Lieber Pierre«, sagte sie dann mit sich ras...                    2  
1    [(Teuerster wird er sagen« … In diesem Augenbl...                    1  
2    [(»Mein geliebtes Herz! Verzeih, wenn ich Dir ...               

## Save the dictionary to a csv file

In [212]:
#save the dataframe segment to a new csv.file

d_prose_texts.to_csv('output_cleaned_letter_openings_subset2_big.csv', index=False)

In [213]:
#show big dataframe 
d_prose_texts

,file_name,document_text,title,clean_text,extracted_matches,match_count,extracted_matches_context,cleaned_matches_context,cleaned_match_count
0,Fontane_Theodor_Cecile.txt,Cécile\n\nErstes Kapitel\n\n»Thale. Zweiter…«\...,Cécile,Cécile Erstes Kapitel »Thale. Zweiter…« »Letzt...,"[(»Lieber Pierre, 28539), (»Lieber Freund, 310...",9,"[(»Lieber Pierre«, sagte sie dann mit sich ras...","[(»Lieber Pierre«, sagte sie dann mit sich ras...",2
1,Hollaender_Felix_Die_Briefe_des_Fraeulein.txt,"Die Briefe des Fräulein Brandt\n\nIserbaude, 7...",Die Briefe des Fräulein Brandt,"Die Briefe des Fräulein Brandt Iserbaude, 7. J...","[(»Liebes Kind, 57567), (Teuerster wird, 16684...",28,"[(»Liebes Kind,« antworte ich trocken, »du bis...",[(Teuerster wird er sagen« … In diesem Augenbl...,1
2,Heyse_Paul_Gegen_den_Strom.txt,Gegen den Strom\n\nErstes Kapitel.\n\nEs war z...,Gegen den Strom,Gegen den Strom Erstes Kapitel. Es war zu Anfa...,"[(Geehrten nun, 349367), (»Mein geliebtes Herz...",3,[(Geehrten nun überreichte. Worauf der alte Ma...,"[(»Mein geliebtes Herz! Verzeih, wenn ich Dir ...",1
3,Federer_Heinrich_Umbrische_Reisegeschichtlein_...,San Benedettos Dornen und San Francescos Rosen...,San Benedettos Dornen und San Francescos Rosen,San Benedettos Dornen und San Francescos Rosen...,[],0,[],[],0
4,Groller_Balduin_Detektiv_Dagobert_Eine_teure_D...,Eine teure Depesche\n\nSie saßen wieder zu dri...,Eine teure Depesche,Eine teure Depesche Sie saßen wieder zu dritt ...,[],0,[],[],0
...,...,...,...,...,...,...,...,...,...
489,Huch_Friedrich_Enzio.txt,"Enzio\n\nEs war ein weites, bequem eingerichte...",Enzio,"Enzio Es war ein weites, bequem eingerichtetes...","[(Mein lieber Sohn!, 227570)]",1,"[(Mein lieber Sohn! sprach sie, ich bin wirkli...","[(Mein lieber Sohn! sprach sie, ich bin wirkli...",1
490,Groller_Balduin_Detektiv_Dagobert_Dagoberts_un...,Dagoberts unfreiwillige Reise\n\nAndreas Grumb...,Dagoberts unfreiwillige Reise,Dagoberts unfreiwillige Reise Andreas Grumbach...,"[(»Lieber Burgholzer, 38565)]",1,"[(»Lieber Burgholzer, Sie müssen doch schon de...","[(»Lieber Burgholzer, Sie müssen doch schon de...",1
491,Ganghofer_Ludwig_Fliegender_Sommer_Die_Liebe_G...,Die Liebe Gottes\n\nWer auf fröhlichcr Sommerr...,Die Liebe Gottes,Die Liebe Gottes Wer auf fröhlichcr Sommerreis...,"[(24. August, 2864)]",1,"[(24. August, am Tage des heiligen Bartholomäu...",[],0
492,Frapan_Ilse_Was_der_Alltag_dichtet_Dort_oben.txt,"Dort oben\n\nJa, Kinder, das ist wohl so, das ...",Dort oben,"Dort oben Ja, Kinder, das ist wohl so, das gla...","[(4. Mai 1842, 15464), (8. Mai, 33581), (8. Ma...",3,"[(4. Mai 1842, und ahnungslos und voll von uns...",[],0


As the dataframe is to huge to have a look at comfortably, we copy the dataframe and limit it to the columns of our interest by deleting the columns we do not need.

In [214]:
#delete the columns from the copy that we do not need or that need to much space
#del d_prose_texts['extracted_matches2']
del d_prose_texts['document_text']
del d_prose_texts['clean_text']
#del d_prose_texts['extracted_letter']
#del d_prose_texts['context_around_letter_openings']

In [215]:
#show small dataframe 
d_prose_texts

,file_name,title,extracted_matches,match_count,extracted_matches_context,cleaned_matches_context,cleaned_match_count
0,Fontane_Theodor_Cecile.txt,Cécile,"[(»Lieber Pierre, 28539), (»Lieber Freund, 310...",9,"[(»Lieber Pierre«, sagte sie dann mit sich ras...","[(»Lieber Pierre«, sagte sie dann mit sich ras...",2
1,Hollaender_Felix_Die_Briefe_des_Fraeulein.txt,Die Briefe des Fräulein Brandt,"[(»Liebes Kind, 57567), (Teuerster wird, 16684...",28,"[(»Liebes Kind,« antworte ich trocken, »du bis...",[(Teuerster wird er sagen« … In diesem Augenbl...,1
2,Heyse_Paul_Gegen_den_Strom.txt,Gegen den Strom,"[(Geehrten nun, 349367), (»Mein geliebtes Herz...",3,[(Geehrten nun überreichte. Worauf der alte Ma...,"[(»Mein geliebtes Herz! Verzeih, wenn ich Dir ...",1
3,Federer_Heinrich_Umbrische_Reisegeschichtlein_...,San Benedettos Dornen und San Francescos Rosen,[],0,[],[],0
4,Groller_Balduin_Detektiv_Dagobert_Eine_teure_D...,Eine teure Depesche,[],0,[],[],0
...,...,...,...,...,...,...,...
489,Huch_Friedrich_Enzio.txt,Enzio,"[(Mein lieber Sohn!, 227570)]",1,"[(Mein lieber Sohn! sprach sie, ich bin wirkli...","[(Mein lieber Sohn! sprach sie, ich bin wirkli...",1
490,Groller_Balduin_Detektiv_Dagobert_Dagoberts_un...,Dagoberts unfreiwillige Reise,"[(»Lieber Burgholzer, 38565)]",1,"[(»Lieber Burgholzer, Sie müssen doch schon de...","[(»Lieber Burgholzer, Sie müssen doch schon de...",1
491,Ganghofer_Ludwig_Fliegender_Sommer_Die_Liebe_G...,Die Liebe Gottes,"[(24. August, 2864)]",1,"[(24. August, am Tage des heiligen Bartholomäu...",[],0
492,Frapan_Ilse_Was_der_Alltag_dichtet_Dort_oben.txt,Dort oben,"[(4. Mai 1842, 15464), (8. Mai, 33581), (8. Ma...",3,"[(4. Mai 1842, und ahnungslos und voll von uns...",[],0


In [216]:
#save the shortened dataframe to a new csv.file

d_prose_texts.to_csv('output_cleaned_letter_openings_subset2_small.csv', index=False)

end of notebook 2